In [21]:
import random
import os

input_dir = '/data/users/sash/stylegan/style-based-gan-pytorch/encoder/raw_images'
input_image = random.choice(os.listdir(input_dir))
input_image_path = input_dir + "/" + input_image
print(input_image)

sasha_portrait_2.jpeg


In [22]:
import sys
sys.path.insert(0, "/data/users/sash/stylegan/style-based-gan-pytorch")

# Getting Alignment

In [23]:
aligned_image_folder = "/data/users/sash/stylegan/style-based-gan-pytorch/encoder/aligned_images_run"

In [24]:
import bz2
from encoder.ffhq_dataset.face_alignment import image_align
from encoder.ffhq_dataset.landmarks_detector import LandmarksDetector

def unpack_bz2(src_path):
    data = bz2.BZ2File(src_path).read()
    dst_path = src_path[:-4]
    with open(dst_path, 'wb') as fp:
        fp.write(data)
    return dst_path


def align_image(input_image_path, aligned_dir):
    """
    Extracts and aligns all faces from images using DLib and a function from original FFHQ dataset preparation step
    python align_images.py /raw_images /aligned_images
    """

    landmarks_model_path = unpack_bz2("/data/users/sash/stylegan/style-based-gan-pytorch/encoder/models/shape_predictor_68_face_landmarks.dat.bz2")
    ALIGNED_IMAGES_DIR = aligned_dir
    img_name = input_image
    landmarks_detector = LandmarksDetector(landmarks_model_path)
    
    for i, face_landmarks in enumerate(landmarks_detector.get_landmarks(input_image_path), start=1):
        face_img_name = '%s_%02d.png' % (os.path.splitext(img_name)[0], i)
        aligned_face_path = os.path.join(ALIGNED_IMAGES_DIR, face_img_name)

        image_align(input_image_path, aligned_face_path, face_landmarks)

align_image(input_image_path, aligned_image_folder)

In [25]:
os.listdir(aligned_image_folder)

['sasha_portrait_2_01.png',
 'sasha_portrait_2_02.png',
 'roman_01.png',
 'zuck_01.png',
 'charles_01.png',
 'sasha_portrait_01.png',
 'pranav_01.png']

# Encoder

In [26]:
sys.path.insert(0, "/data/users/sash/stylegan/style-based-gan-pytorch/encoder/pytorch_stylegan_encoder")

In [27]:
aligned_image_name = [x for x in os.listdir(aligned_image_folder) if os.path.splitext(input_image)[0] in x][0]
aligned_image_name_wo_extention = os.path.splitext(aligned_image_name)[0]
aligned_image_path = aligned_image_folder + "/" + aligned_image_name
print(aligned_image_name)

sasha_portrait_2_01.png


In [28]:
latents_path = "/data/users/sash/stylegan/style-based-gan-pytorch/encoder/pytorch_stylegan_encoder/latents/"
latent_file_path = latents_path + aligned_image_name_wo_extention +".npy"
optimized_folder_path = "/data/users/sash/stylegan/style-based-gan-pytorch/encoder/pytorch_stylegan_encoder/optimized/"

In [29]:
import os
os.path.splitext(os.path.basename(latent_file_path))[0]

'sasha_portrait_2_01'

In [30]:
from encoder.pytorch_stylegan_encoder.encode_image import main as encode_image

args = [
    "--save_optimized_image",
    "True",
    "--image_path",
    aligned_image_path,
    "--dlatent_path",
    latent_file_path,
]

encode_image(args)
optimized_img_path = "/data/users/sash/stylegan/style-based-gan-pytorch/optimized.png"
os.rename(optimized_img_path, optimized_folder_path + "optimized_" + aligned_image_name)

Optimizing Latents.
[2020-03-30 22:20:04,863][INFO] Loading pytorch model from `encoder/pytorch_stylegan_encoder/InterFaceGAN/models/pretrain/stylegan_ffhq.pth`.
[2020-03-30 22:20:04,978][INFO] Successfully loaded!
[2020-03-30 22:20:04,979][INFO]   `lod` of the loaded model is 0.0.


Step: 999, Loss: 1.8740954399108887: 100%|██████████| 1000/1000 [03:34<00:00,  4.66it/s]


In [10]:
%tb

No traceback available to show.


In [11]:
print(os.listdir(latents_path))
print(os.listdir(optimized_folder_path))

['sasha_portrait_2_01.npy', 'roman_01.npy', 'pranav_01.npy']
['optimized_sasha_portrait_2_01.png', 'optimized_roman_01.png', 'optimized_pranav_01.png']


# StyleMixing with WP

In [12]:
import numpy as np
import torch

In [13]:
wp_path = latents_path + aligned_image_name_wo_extention + ".npy"
other_name = [f for f in os.listdir(latents_path) if aligned_image_name_wo_extention not in f][0]
wp_other_path = latents_path + other_name
morph_interpolation_result_folder = "/data/users/sash/stylegan/style-based-gan-pytorch/encoder/pytorch_stylegan_encoder/morphed_intepolation/"
morphed_img_name = aligned_image_name_wo_extention + "_" + other_name[:-4]
print(wp_path)
print(wp_other_path)
print(morph_interpolation_result_folder + morphed_img_name + ".png")

/data/users/sash/stylegan/style-based-gan-pytorch/encoder/pytorch_stylegan_encoder/latents/pranav_01.npy
/data/users/sash/stylegan/style-based-gan-pytorch/encoder/pytorch_stylegan_encoder/latents/sasha_portrait_2_01.npy
/data/users/sash/stylegan/style-based-gan-pytorch/encoder/pytorch_stylegan_encoder/morphed_intepolation/pranav_01_sasha_portrait_2_01


In [17]:
args = [
    "--morph_interpolation",
    "True",
    "--latent_path",
    wp_path,
    "--other_latent_path",
    wp_other_path,
    "--num_images",
    "4",
    "--morph_interpolation_image_path",
    morph_interpolation_result_folder + morphed_img_name + ".png"
]

encode_image(args)

Synthesizing Morphing Images
Interpolated wps: torch.Size([6, 1, 18, 512])
[2020-03-28 15:58:24,664][INFO] Loading pytorch model from `encoder/pytorch_stylegan_encoder/InterFaceGAN/models/pretrain/stylegan_ffhq.pth`.
[2020-03-28 15:58:24,772][INFO] Successfully loaded!
[2020-03-28 15:58:24,772][INFO]   `lod` of the loaded model is 0.0.
[2020-03-28 15:58:25,187][INFO] Loading pytorch model from `encoder/pytorch_stylegan_encoder/InterFaceGAN/models/pretrain/stylegan_ffhq.pth`.
[2020-03-28 15:58:25,306][INFO] Successfully loaded!
[2020-03-28 15:58:25,306][INFO]   `lod` of the loaded model is 0.0.
[2020-03-28 15:58:25,709][INFO] Loading pytorch model from `encoder/pytorch_stylegan_encoder/InterFaceGAN/models/pretrain/stylegan_ffhq.pth`.
[2020-03-28 15:58:25,825][INFO] Successfully loaded!
[2020-03-28 15:58:25,826][INFO]   `lod` of the loaded model is 0.0.
[2020-03-28 15:58:26,224][INFO] Loading pytorch model from `encoder/pytorch_stylegan_encoder/InterFaceGAN/models/pretrain/stylegan_ffhq.